In [60]:
# import pickle
# from plm_special.data.exp_pool import ExperiencePool

# # 打开并读取 pickle 文件
# with open('artifacts/exp_pools/exp_pool.pkl', "rb") as f:
#     exp_pool = pickle.load(f)

# # 输出数据内容
# # print(dir(exp_pool))
# # 查看对象的字典表示
# print(vars(exp_pool))

In [61]:
def min_max_normalize(data):
    min_val = min(data)
    max_val = max(data)
    return [(x - min_val) / (max_val - min_val) for x in data]

In [65]:
import os

llm_path = "llama_base-lora/early_stop_-1_rank_128_w_20_gamma_1.0_tgt_scale_1.0_seed_100003"
mpc_path = "mpc/seed_100003"
genet_path = "genet/seed_100003"
bba_path ='bba/seed_100003'

dir_path =[llm_path,mpc_path,genet_path,bba_path]
colums = [
    'time_stamp',
    'bit_rate',
    'buffer_size',
    'rebuf',
    'video_chunk_size',
    'download_time',
    'smoothness',
    'reward',
]

In [68]:
for dir in dir_path:
    file_path = dir
    print('方法:',file_path.split('_')[0])
    qoe_list = []
    for file in os.listdir(file_path):
        with open(os.path.join(file_path, file), "r") as f:
            qoe = 0
            C = 0  # 统计有效的分段数量
            for line in f:  # 改为逐行读取，减少内存使用
                line = line.strip().split("\t")
                if len(line) < 4:  # 检查行是否有效
                    continue
                try:
                    # 提取字段并转换为浮点数
                    bitrate = float(line[1])
                    rebu = float(line[3])
                    smo = float(line[-2])
                    # 累计 QoE 和分段计数
                    qoe += bitrate - 4.3 * rebu - smo
                    C += 1  # 有效行数 +1
                except ValueError:
                    # 如果某一行无法转换为数字，则跳过
                    continue
            # 计算平均 QoE（避免 C 为 0 的情况）
            if C > 0:
                average_qoe = qoe / C
                qoe_list.append(average_qoe)
                # print(f"Average QoE for file {file}: {average_qoe}")
            else:
                print(f"No valid data in file {file}")
    qoe_list= min_max_normalize(qoe_list)
    # print(qoe_list)
    print(sum(qoe_list) / len(qoe_list))

方法: llama
0.30069394057269333
方法: mpc/seed
0.2994409295042655
方法: genet/seed
0.3076460287766969
方法: bba/seed
0.30652793806307094
